In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

In [2]:
d2020=pd.read_csv(r"data\Data\2020\imdb_movies_2020.csv")
d2021=pd.read_csv(r"data\Data\2021\imdb_movies_2021.csv")
d2022=pd.read_csv(r"data\Data\2022\imdb_movies_2022.csv")
d2023=pd.read_csv(r"data\Data\2023\imdb_movies_2023.csv")
d2024=pd.read_csv(r"data\Data\2024\imdb_movies_2024.csv")

data=pd.concat([d2020,d2021,d2022,d2023,d2024])


In [7]:
print(len(data))
data= data.sample(frac=1).reset_index()
data.head()

2850


,index,Title,Movie Link,Year,Duration,MPA,Rating,Votes
0,228,229. Waltair Veerayya,https://www.imdb.com/title/tt15281402/?ref_=sr...,2023,2h 35m,NaN,6.2,7.9K
1,520,521. Sharper,https://www.imdb.com/title/tt12573454/?ref_=sr...,2023,1h 56m,R,6.7,34K
2,536,537. Lamborghini: The Man Behind the Legend,https://www.imdb.com/title/tt5533370/?ref_=sr_...,2022,1h 37m,R,5.5,15K
3,460,461. Year 10,https://www.imdb.com/title/tt21154574/?ref_=sr...,2024,1h 36m,NaN,4.7,565
4,106,107. K.G.F: Chapter 2,https://www.imdb.com/title/tt10698680/?ref_=sr...,2022,2h 46m,Not Rated,8.2,156K


In [ ]:
# data.to_csv('movie_data.csv')

In [43]:
data = pd.read_csv('movie_data.csv')

In [44]:
data.head()

,Unnamed: 0,index,Title,Movie Link,Year,Duration,MPA,Rating,Votes
0,0,228,229. Waltair Veerayya,https://www.imdb.com/title/tt15281402/?ref_=sr...,2023,2h 35m,NaN,6.2,7.9K
1,1,520,521. Sharper,https://www.imdb.com/title/tt12573454/?ref_=sr...,2023,1h 56m,R,6.7,34K
2,2,536,537. Lamborghini: The Man Behind the Legend,https://www.imdb.com/title/tt5533370/?ref_=sr_...,2022,1h 37m,R,5.5,15K
3,3,460,461. Year 10,https://www.imdb.com/title/tt21154574/?ref_=sr...,2024,1h 36m,NaN,4.7,565
4,4,106,107. K.G.F: Chapter 2,https://www.imdb.com/title/tt10698680/?ref_=sr...,2022,2h 46m,Not Rated,8.2,156K


In [45]:
data.drop([ 'Unnamed: 0'], inplace=True,axis=1)


In [46]:
def convert_duration(duration):
    if isinstance(duration, str) and "h" in duration:
        parts = duration.split()
        hours = int(parts[0][0]) * 60  
        minutes = int(parts[1][:-1]) if len(parts) > 1 else 0  
        return hours + minutes
    return None

data['Duration']=data['Duration'].apply(convert_duration)


In [47]:
def convert_votes(votes):
    if isinstance(votes, str):
        if "K" in votes:
            return int(float(votes.replace("K", "")) * 1000)
        return int(votes.replace(",", ""))  
    return None



In [48]:
data["Votes"] = data["Votes"].apply(convert_votes)


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850 entries, 0 to 2849
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       2850 non-null   int64  
 1   Title       2850 non-null   object 
 2   Movie Link  2850 non-null   object 
 3   Year        2850 non-null   int64  
 4   Duration    2822 non-null   float64
 5   MPA         1949 non-null   object 
 6   Rating      2837 non-null   float64
 7   Votes       2837 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 178.2+ KB


In [50]:
data.isna().sum()

index           0
Title           0
Movie Link      0
Year            0
Duration       28
MPA           901
Rating         13
Votes          13
dtype: int64

In [51]:
data["Duration"].fillna(data["Duration"].median(), inplace=True)
data["Rating"].fillna(data["Rating"].median(), inplace=True)
data["Votes"].fillna(data["Votes"].median(), inplace=True)
data["MPA"].fillna("Unknown", inplace=True)

In [71]:
data.MPA.unique()

array(['Unknown', 'R', 'Not Rated', 'TV-MA', 'PG-13', 'PG', 'TV-14',
       'Unrated', 'TV-PG', 'TV-G', 'G', '18+', 'TV-Y7-FV', 'TV-Y7', '16+',
       'Approved', '13+', 'NC-17', 'TV-Y'], dtype=object)

In [52]:
data.head()

,index,Title,Movie Link,Year,Duration,MPA,Rating,Votes
0,228,229. Waltair Veerayya,https://www.imdb.com/title/tt15281402/?ref_=sr...,2023,155.0,Unknown,6.2,7900.0
1,520,521. Sharper,https://www.imdb.com/title/tt12573454/?ref_=sr...,2023,116.0,R,6.7,34000.0
2,536,537. Lamborghini: The Man Behind the Legend,https://www.imdb.com/title/tt5533370/?ref_=sr_...,2022,97.0,R,5.5,15000.0
3,460,461. Year 10,https://www.imdb.com/title/tt21154574/?ref_=sr...,2024,96.0,Unknown,4.7,565.0
4,106,107. K.G.F: Chapter 2,https://www.imdb.com/title/tt10698680/?ref_=sr...,2022,166.0,Not Rated,8.2,156000.0


In [54]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder

In [58]:
encoder = OneHotEncoder(sparse=False)
MPA_encoded = encoder.fit_transform(data[["MPA"]])
year_reshaped = data['Year'].values.reshape(-1, 1)
duration_reshaped = data['Duration'].values.reshape(-1, 1)
rating_reshaped = data['Rating'].values.reshape(-1, 1)
X = np.hstack([year_reshaped, duration_reshaped, MPA_encoded, rating_reshaped])

knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(X)

c:\Users\mahmo\anaconda3\envs\workDeep\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


NearestNeighbors(metric='cosine')

In [70]:
import joblib
joblib.dump(knn,'knn.pkl')
joblib.dump(encoder,'encoder.pkl')

['encoder.pkl']

In [59]:
q={
    'Year':2020,
    'Duration':150,
    'MPA':'R',
    'Rating':8.5
}

In [ ]:

def get_similar_items(q, data, knn, encoder):
    q_mpa_encoded = encoder.transform([[q['MPA']]])
    
    q_year = np.array([[q['Year']]])
    q_duration = np.array([[q['Duration']]])
    q_rating = np.array([[q['Rating']]])
    
    q_vector = np.hstack([q_year, q_duration, q_mpa_encoded, q_rating])
    
    distances, indices = knn.kneighbors(q_vector)
    
    similar_items = data.iloc[indices[0]].to_dict(orient='records')
    return similar_items
   



In [66]:
similar_items = get_similar_items(q, data, knn, encoder)


c:\Users\mahmo\anaconda3\envs\workDeep\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [68]:
similar_items[0]

{'index': 123,
 'Title': '124. Anatomy of a Fall',
 'Movie Link': 'https://www.imdb.com/title/tt17009710/?ref_=sr_t_124',
 'Year': 2023,
 'Duration': 151.0,
 'MPA': 'R',
 'Rating': 7.7,
 'Votes': 163000.0}

In [69]:
similar_items[1]

{'index': 151,
 'Title': '152. Coldplay Music of The Spheres Live Broadcast from Buenos Aires',
 'Movie Link': 'https://www.imdb.com/title/tt23016388/?ref_=sr_t_152',
 'Year': 2022,
 'Duration': 150.0,
 'MPA': 'Unknown',
 'Rating': 8.4,
 'Votes': 93.0}